This is a follow up on my previous kernel about the structure in missing values and whether one can make use of it. You can find the previous kernel [here](https://www.kaggle.com/cgump3rt/two-sigma-financial-modeling/investigate-missing-values). The short summary from the training data is:

* fraction of missing data can is significant for many feature columns,
* NaNs are **not** distributed randomly per feature but span a continuous range in time
* all NaNs for an asset ID appear at the beginning of the period during which this asset is part of the portfolio.

I saw that there are already quite a few interesting kernels by other people who also tried to cluster assets based on the NaN structure. For example have a look at the following kernels if you are interested (list non-exhaustive)

* by [jrisk123](https://www.kaggle.com/jriskas/two-sigma-financial-modeling/asset-cluster-visualization)
* by [lesibius](https://www.kaggle.com/lesibius/two-sigma-financial-modeling/financial-instrument-types)
* by [chbimo](https://www.kaggle.com/chbimo/two-sigma-financial-modeling/correlation-of-nan-structure-between-id)
* by [Allunia](https://www.kaggle.com/allunia/two-sigma-financial-modeling/feature-dynamics-looking-at-id-groups)
* by [reziproke](https://www.kaggle.com/reziproke/two-sigma-financial-modeling/nan-structure-on-id-level-eda)
* (sorry if I missed you, add comment and I will update the list...)

Before giving the clusterization yet another try, something came to my mind when I was skiing last week. What if data is published in fixed intervals (e.g. every working day at 9am, or Mondays, or the first Tuesday of the month)? In this scenario, the NaN structure does not necessarily characterize different asset categories, but also depends when an asset was bought relative to the next _publication date_. Imagine data were to be published always on Mondays. Then the number of missing NaN values would probably tell you which weekday you bough this asset.

Long story short: let's see whether we can find some pattern in the total number of NaN values per timestamp.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
%matplotlib inline

In [ ]:
with pd.HDFStore('../input/train.h5') as train:
    df = train.get('train')

As a first step, we get a new data frame containing the total number of NaN values for each column and timestamp. Since we are not that much interested in the total numbers, but rather in how they change, we take the difference between two subsequent timestamps. Finally, we make a plot showing the evolution of the total number of NaNs.

In [ ]:
# get total number of NaN values per feature column for every timestamp
nans = df.groupby('timestamp').apply(lambda x: x.isnull().sum())
# get change with respect 
nans = nans.diff().drop(['id','timestamp','y'],axis=1)
nans.plot(figsize=(13,10))
plt.legend(ncol=5,loc='best');

Voila, at a first glance, it looks that data becomes available in regular intervals. It is interesting to see that there huge spikes in the beginning while later on, the magnitude becomes much smaller. But please not that these are absolute numbers. So the total number of assets in the portfolio is important. My guess is that we start with a large number of assets at timestamp 0. Therefore, we have a lot of NaNs which get filled over time. Over the training period, only comparably small chunks of assets are added leading to a smaller number of NaNs which gets filled later on.  
There are also few timestamps at which the number of NaNs in the dataset increases. My suspicion is that these peaks occur at times when new assets are bought. So let's check this quickly.

In [ ]:
# get the number of assets per timestamp
nassets = df.groupby('timestamp').apply(len)
# get the change with respect to the previous timestamp
delta_assets = nassets.diff()

# plot again
nans.plot(figsize=(13,10))
delta_assets.plot(style=['.b'],ax=plt.gca(),label='number of assets')
plt.legend(ncol=5,loc='best');

Ok, so increases in the number of NaNs are explained by new assets being bought. Just out of curiosity I made a plot summing the number of NaNs for the different groups of feature columns (_fundamental_, _derived_ and _technical_). This time I divide by the number of assets present and also by the number of feature columns in each group. Therefore, the numbers can be interpreted as average probability that a fundamental/derived/technical feature column has a NaN at each timestamp.

In [ ]:
# get number of columns for each group
n_fundamental = len([g for g in nans.columns.tolist() if re.search("fundamental_*",g)])
n_derived = len([g for g in nans.columns.tolist() if re.search("derived_*",g)])
n_technical = len([g for g in nans.columns.tolist() if re.search("technical_*",g)])

In [ ]:
nans['fundamental'] = nans.filter(regex='fundamental_*').sum(axis=1)/nassets/n_fundamental
nans['derived'] = nans.filter(regex='derived_*').sum(axis=1)/nassets/n_derived
nans['technical'] = nans.filter(regex='technical_*').sum(axis=1)/nassets/n_technical
nans[['fundamental','derived','technical']].plot(figsize=(12,6));

In this graph one can observe a pattern which was already present in the other plot above, but harder to see. While information updates seem to be published regurlarly, one can still observe a pattern which correlates with the dates at which assets are bought. It looks like every time new assets are bought (introducing more NaNs), a few timestamps later this is reverted. We can try to find the _lag_ of this anti-correlation to get a feeling for the number of NaNs we can expect for each asset in the beginning.  
Just to be clear on what the next code is doing (I know it's easy to get lost ;-)):
* We have the change in the number of assets between to timestamps (`delta_assets`).
* We have the change in the total number of NaNs for each feature column and timestamp (`nans`).  

I am going to shift `delta_assets` for different lag values and calculate the correlation for each feature column.

In [ ]:
max_lag = 100
corrs = np.zeros((max_lag,nans.shape[1]))
for l in range(1,max_lag):
    c = delta_assets.shift(l)
    corrs[l] = nans.corrwith(c)

plt.figure(figsize=(15,9))
sns.heatmap(corrs,xticklabels=nans.columns.tolist(),linewidths=0.05,linecolor='gray')
plt.ylabel('lag');

So, let's interpret the plot above. Derived and fundamental features (if not present from the beginning) become available after 3 timestamps as do most of the technical features as well. Some technical features, however, take much longer than 3 time stamps. For example, technical_1 seems to become available after 15 timestamps. It's an anti-correlation because the number of assets goes up and few days later the number of NaNs goes down again. Even though, I am not sure how useful this it, I found it interesting how one can easily learn something about the _nature_ of the different technical features (which, of course, is kind of expected).

Turning to clusterization. The first question is, of course, what kind of data we should use as features during the clusterization. Most attempts I have seen so far were using the fraction of timestamps for which a given feature column was not filled with data. In my opinion, this is non-optimal choice for the following reason:
1. I expected the NaN patterns are intrinsically related to type of the asset.
1. They are independent of the time the asset is kept in the portfolio.
1. They are driven by external or technical constraints (e.g. publication of information at given dates, technical indicators requiring a decent amount of history etc).

Especially point 2) makes the usage of relative fractions sub-optimal (it might be okey if assets are kept for ~the same time). Using the absolute number of timestamps for which a given feature is not available is not optimal either. If an asset is kept for 1000 timestamps, is it a real difference if the data is not available for the first 3 or first 5 days? My suggestion is kind of hybrid solution: I will only consider the first 100 timestamps for each asset and take the relative fraction of non-availability in these first 100 timestamps (which of course is only acting like a global scaling factor). The underlying assumption is here that the NaN structure during the first 100 timestamps provides sufficient information for distinguishing different asset categories. I have to admit that the choice of 100 is completely random at this point. So here is how it looks in code:

In [ ]:
# group by asset ID and get number of missing values for each feature column
ids = sorted(df.id.unique())
columns = df.columns.drop(['id','timestamp','y']).insert(0,'length')
nan_df = pd.DataFrame(data=None,index=ids,columns=columns,dtype=float)
# iterate over all asset ID
for name,group in df.groupby('id'):
    # for every feature column
    for c in columns:
        if c == 'length':
            nan_df.loc[name,c] = int(len(group))
        else:
            # total number of rows with missing data
            nan_df.loc[name,c] = float(group[c].isnull().sum())

In [ ]:
nan_df.head()

In [ ]:
# truncate all numbers at 100
capped = nan_df.copy()
capped[capped > 100] = 100
capped = capped.div(capped['length'],axis='index').drop(['length'],axis=1)
capped.head()

In [ ]:
from sklearn.cluster import dbscan
_,labels = dbscan(capped.values)
plt.hist(labels,bins=25,range=(-0.5,24.5));

At the first attempt it looks like there are three quite prominent categories while the others are probably all outliers. I think this is roughly inline with the results reported by others that the number of clusters should be around 5. One should make a more thorough test of the clusterisation result, also playing with the parameters of DBScan. If you want to have a go, just fork the notebook. The next step would then be to see whether assets of different clusters actually behave differently (e.g. different correlation between features and target values etc).